# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Общая информация о данных

In [1]:
import pandas as pd
from pymystem3 import Mystem
m = Mystem()

data = pd.read_csv('data.csv')
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В таблице 12 столбцов.

Согласно документации к данным:
* `children` — количество детей в семье
* `days_employed` — общий трудовой стаж в днях
* `dob_years` — возраст клиента в годах
* `education` — уровень образования клиента
* `education_id` — идентификатор уровня образования
* `family_status` — семейное положение
* `family_status_id` — идентификатор семейного положения
* `gender` — пол клиента
* `income_type` — тип занятости
* `debt` — имел ли задолженность по возврату кредитов
* `total_income` — ежемесячный доход
* `purpose` — цель получения кредита

В названиях столбцов нарушений нет.

Типы данных в столбцах целочисленные, вещественные и строковые. 

В данных есть пропущенные значения - количество значений в стобцах `days_employed` и `total_income` меньше остальных столбцов.

**Вывод**

В каждой строке таблицы содержится информация о клиенте банка. На первый взляд, данных хватает для проверки гипотез. Однако после первой полученной информации о таблице стали видны пропущеные значения, дубликаты, отрицательные значения и другие проблемы - для начала необходимо с ними разобраться. 

## Шаг 2. Предобработка данных

### Обработка пропусков

Посмотрим на количество и расположение пропущенных значений.

In [3]:
data.isna().sum() 

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [4]:
data[data['days_employed'].isna()].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


Для начала нужно разобраться с корректностью данных в интересующих столбцах.
Так как большинство значений в столбце 'days_employed' отрицальные, то можно предположить, что они верные, но внесены в систему некорректно. Поэтому 'минусы' можно просто убрать, с помощью модуля.

In [5]:
data['days_employed'] = abs(data['days_employed'])
data['total_income'] = abs(data['total_income'])

Также в этом столбце заметны значения сильно больше остальных. Сперва попробуем перевести дни стажа в года. 

In [6]:
data['days_employed'] = data['days_employed']/365
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,23.116912,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,11.026860,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,15.406637,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,11.300677,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,932.235814,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,2.537495,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,7.888225,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,0.418574,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,18.985932,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,5.996593,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Стало явно видно, что некоторые значения не соответвуют реальности. Я предполагаю, что эти аномальные значения были внесены в систему в часах.

In [7]:
data.loc[data['days_employed'] > 100, 'days_employed'] = data.loc[data['days_employed'] > 100, 'days_employed'] / 24
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,23.116912,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,11.026860,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,15.406637,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,11.300677,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,38.843159,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,2.537495,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,7.888225,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,0.418574,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,18.985932,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,5.996593,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Теперь данные в столбцах выглядят релистично.

Для того, чтобы как можно меньше повлиять на результаты исследования, заполняю пропущеные количественные переменные медианами в зависимости от типа занятости.

In [8]:
def median_filling (list_): #функция, которая принимает список значений, отфильтрованный по уникальному типу занятости, и заполнияет в нем пропуски медианой
    
    for income in data['income_type'].unique():
        median = data.loc[data['income_type'] == income, list_].median()
        data.loc[(data[list_].isna()) & (data['income_type'] == income), list_] = median
for column in ['days_employed', 'total_income']: # применение функции для столбцов, в которых были обнаружены пропуски
    median_filling(column)
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Вывод**

С пропусками в данных справились.

### Замена типа данных

Значения в стобцах 'days_employed' и 'total_income' должны быть целочисленными, поэтому их необходимо перевести в тип *int* методом *astype()*.

In [9]:
data[['days_employed', 'total_income']] = data[['days_employed', 'total_income']].astype('int')
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,23,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,11,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,15,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,11,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,38,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


**Вывод**

С количественными переменными справились, перейдем к категориальным.

### Обработка дубликатов

Пройдемся по всем столбцам для того, чтобы не упустить проблемы в данных.

In [10]:
cols = ['children', 'education', 'family_status', 'gender', 'income_type']
for column in cols:
    display(data[column].unique())

array([ 1,  0,  3,  2, -1,  4, 20,  5])

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

array(['F', 'M', 'XNA'], dtype=object)

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

В столбце содержащем количество детей в семье есть отрицальное значение '-1', поэтому применим функцию модуля ко всему столбцу.

In [11]:
data['children'] = abs(data['children'])
data['children'].unique()

array([ 1,  0,  3,  2,  4, 20,  5])

In [12]:
data[data['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,6,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905,покупка недвижимости


Нашлось одно странное значение в столбце 'gender', но так как оно не повлияет на результаты исследования, я не вижу смысла удалять эту строку из таблицы.

В столбце 'education' есть дубликаты, которые отличаются регистром. Для приведения их к нижнему регистру в библиотке Pandas есть специльный метод *str.lower()*.

In [13]:
data['education'] = data['education'].str.lower()
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

С остальными столбцами всё в порядке, дубликатов нет.

Далее необходимо проверить таблицу на наличие полных дубликатов, то есть строк в которых все значения совпадают. Чтобы их увидеть нужно таблицу с дубликатами отсортировать по всему набору колонок.

In [14]:
data[data.duplicated(keep = False)].sort_values(by = list(data.columns))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
15892,0,4,23,среднее,1,Не женат / не замужем,4,F,сотрудник,0,142594,сделка с подержанным автомобилем
19321,0,4,23,среднее,1,Не женат / не замужем,4,F,сотрудник,0,142594,сделка с подержанным автомобилем
3452,0,4,29,высшее,0,женат / замужем,0,M,сотрудник,0,142594,покупка жилой недвижимости
18328,0,4,29,высшее,0,женат / замужем,0,M,сотрудник,0,142594,покупка жилой недвижимости
4216,0,4,30,среднее,1,женат / замужем,0,M,сотрудник,0,142594,строительство жилой недвижимости
...,...,...,...,...,...,...,...,...,...,...,...,...
9238,2,4,34,среднее,1,женат / замужем,0,F,сотрудник,0,142594,покупка жилья для сдачи
11033,2,4,39,среднее,1,гражданский брак,1,F,сотрудник,0,142594,сыграть свадьбу
16902,2,4,39,среднее,1,гражданский брак,1,F,сотрудник,0,142594,сыграть свадьбу
9013,2,7,36,высшее,0,женат / замужем,0,F,госслужащий,0,150447,получение образования


Строк-дубликатов в таблице оказалось немало, а именно 137. Удалим дубликаты и посмотрим на итоговую сумму дубликатов.

In [15]:
data = data.drop_duplicates().reset_index(drop = True) 

In [16]:
data.duplicated().sum()

0

**Вывод**

Дубликаты в данных обработаны.

### Лемматизация

Для того, чтобы выделить категории среди целей получения кредита, необходимо лемматизировать данные, то есть привести слова к их словарной форме.

In [17]:
lemmas = m.lemmatize(' '.join(data['purpose']))
from collections import Counter
print(Counter(lemmas)) 

Counter({' ': 55023, 'недвижимость': 6351, 'покупка': 5897, 'жилье': 4460, 'автомобиль': 4306, 'образование': 4013, 'с': 2918, 'операция': 2604, 'свадьба': 2324, 'свой': 2230, 'на': 2222, 'строительство': 1878, 'высокий': 1374, 'получение': 1314, 'коммерческий': 1311, 'для': 1289, 'жилой': 1230, 'сделка': 941, 'дополнительный': 906, 'заниматься': 904, 'подержать': 853, 'проведение': 768, 'сыграть': 765, 'сдача': 651, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'приобретение': 461, 'профильный': 436, 'подержанный': 111, '\n': 1})


В полученных результатах можно выделить 4 крупные категории: недвижимость/жилье, автомобиль, образование, свадьба.

In [18]:
def purpose_change(purpose):
    lemmas_row = m.lemmatize(purpose)
    if 'жилье' in lemmas_row or 'недвижимость' in lemmas_row:
            return 'недвижимость'
    if 'автомобиль'in lemmas_row:
        return 'автомобиль'
    if 'образование' in lemmas_row:
        return 'образование'
    if 'свадьба' in lemmas_row:
        return 'свадьба'
    return 'иное'
        
data['purpose_new']= data['purpose'].apply(purpose_change)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_new
0,1,23,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,11,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,15,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,11,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,38,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба


**Вывод**

В таблицу был добавлен столбец с выделенной категорией целей получения кредита.  

### Категоризация данных

Для того, чтобы ответить на поставленные вопросы ещё нужно категоризировать уровень дохода и наличие детей.
Доход я разделила на три группы: 'низкий', 'средний', 'высокий', а наличие детей на две: 'есть дети', 'нет детей'.

In [19]:
def total_income_group(income):
    if income <= 120000:
        return 'низкий'
    if 120000 < income < 170000:
        return 'средний'
    if income >= 170000:
        return 'высокий'
data['total_income_group'] = data['total_income'].apply(total_income_group)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_new,total_income_group
0,1,23,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,высокий
1,1,11,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,низкий
2,0,15,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,средний
3,3,11,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,высокий
4,0,38,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,средний


In [20]:
def children_group(children):
    if children == 0:
        return 'нет детей'
    if children >= 0:
        return 'есть дети'
data['children_group'] = data['children'].apply(children_group)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_new,total_income_group,children_group
0,1,23,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,высокий,есть дети
1,1,11,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,низкий,есть дети
2,0,15,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,средний,нет детей
3,3,11,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,высокий,есть дети
4,0,38,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,средний,нет детей


Данные в таблице уже были категоризированы по семейному положению и образованию. Ниже приведены словари:

In [21]:
family_status_dict = data[['family_status_id', 'family_status']].drop_duplicates().reset_index(drop=True)
family_status_dict.head(10)

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


In [22]:
education_dict = data[['education_id', 'education']].drop_duplicates().reset_index(drop=True)
education_dict.head(10)

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


**Вывод**

Проблемы в данных устранены и данные категоризированы. Можно приступать к ответам на вопросы.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [23]:
data_grouped_children = data.pivot_table(index=['children_group'], values='debt', aggfunc=['sum', 'count', 'mean'])
data_grouped_children.columns = ['debt', 'total', '%']
data_grouped_children = data_grouped_children.sort_values(by = '%', ascending = False)
data_grouped_children.style.format({'%': '{:.2%}'})

,debt,total,%
children_group,,,
есть дети,678,7363,9.21%
нет детей,1063,14091,7.54%


**Вывод**

Процент клиентов, имеющих задолженность по кредитам, среди тех, у кого есть дети, выше, чем у клиентов, у которых нет детей.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [24]:
data_grouped_family_status = data.pivot_table(index=['family_status'], values='debt', aggfunc=['sum', 'count', 'mean'])
data_grouped_family_status.columns = ['debt', 'total', '%']
data_grouped_family_status = data_grouped_family_status.sort_values(by = '%', ascending = False)
data_grouped_family_status

,debt,total,%
family_status,,,
Не женат / не замужем,274,2810,0.097509
гражданский брак,388,4151,0.093471
женат / замужем,931,12339,0.075452
в разводе,85,1195,0.071130
вдовец / вдова,63,959,0.065693


**Вывод**

Процент клиентов, имеющих задолженность по кредитам, среди клиентов никогда не состоящих в официальном браке, выше, чем у клиентов, которые состоят или состояли в официальном браке.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [25]:
data_grouped_total_income = data.pivot_table(index=['total_income_group'], values='debt', aggfunc=['sum', 'count', 'mean'])
data_grouped_total_income.columns = ['debt', 'total', '%']
data_grouped_total_income = data_grouped_total_income.sort_values(by = '%', ascending = False)
data_grouped_total_income

,debt,total,%
total_income_group,,,
средний,578,6505,0.088855
низкий,586,7231,0.081040
высокий,577,7718,0.074760


**Вывод**

По полученным результатам, клиенты с высоким доходом имеют самый маленький процент задолженностей, у клиентов с низким доходом этот процент чуть выше, а клиенты со средним доходом имеют самый большой процент задолженностей. Однако процентная разница довольно мала.

- Как разные цели кредита влияют на его возврат в срок?

In [26]:
data_grouped_purpose = data.pivot_table(index=['purpose_new'], values='debt', aggfunc=['sum', 'count', 'mean'])
data_grouped_purpose.columns = ['debt', 'total', '%']
data_grouped_purpose = data_grouped_purpose.sort_values(by = '%', ascending = False)
data_grouped_purpose

,debt,total,%
purpose_new,,,
автомобиль,403,4306,0.093590
образование,370,4013,0.092200
свадьба,186,2324,0.080034
недвижимость,782,10811,0.072334


Кредиты, взятые на автомобили и образование, возвращают хуже, чем кредиты на свадьбы и недвижимость. Недвижимость - самая надежная категория.

Дополнительно можно посмотреть есть ли зависимость между образованием и возвратом кредита в срок, а также между типом занятости и возвратом кредита в срок:

In [27]:
data_grouped_education = data.pivot_table(index=['education'], values='debt', aggfunc=['sum', 'count', 'mean'])
data_grouped_education.columns = ['debt', 'total', '%']
data_grouped_education = data_grouped_education.sort_values(by = '%', ascending = False)
data_grouped_education

,debt,total,%
education,,,
начальное,31,282,0.109929
неоконченное высшее,68,744,0.091398
среднее,1364,15172,0.089902
высшее,278,5250,0.052952
ученая степень,0,6,0.000000


**Вывод**

Здесь прослеживается зависимость процента задолженностей от уровня образования клиентов. Чем выше уровень образования, тем ниже процент зодолженностей по кредитам.

In [28]:
data_grouped_income_type = data.pivot_table(index=['income_type'], values='debt', aggfunc=['sum', 'count', 'mean'])
data_grouped_income_type.columns = ['debt', 'total', '%']
data_grouped_income_type = data_grouped_income_type.sort_values(by = '%', ascending = False)
data_grouped_income_type

,debt,total,%
income_type,,,
в декрете,1,1,1.000000
безработный,1,2,0.500000
сотрудник,1061,11084,0.095724
компаньон,376,5078,0.074045
госслужащий,86,1457,0.059025
пенсионер,216,3829,0.056412
предприниматель,0,2,0.000000
студент,0,1,0.000000


**Вывод**

Госслужащие и пенсионеры имеют меньше задолженностей по кредитам, чем сотрудники и компаньоны. 
У клиентов, находящихся в декрете, не имеющих работы, предпринимателей и студентов невозможно проследить зависимость, так как в данных слишком маленькая выборка.

## Шаг 4. Общий вывод

В процессе исследавания предоставленные данные были обработаны. В некоторых столбцах были обнаружены данные в неверном формате (дни стажа в часах). Пропуски в ежемесячном доходе и трудовом стаже были заполнены медианными значениями в зависимости от типа занятости клиента. Были найдены отрицательные значения и взяты по модулю. Такие пробелмы могли произойти при объединения данных из разных систем.
Данные были разделены на категории, причины взятия кредита были лемматизированы для удобства представления и дальнейшего анализа.

**Рекомендации**

Прошу обратить внимание на следущие проблемы в данных:
- пропуски в столбцах 'total_income' и 'days_employed'
- отрицательные значения в столбцах 'total_income' и 'children'
- вещественный тип данных в столбцах 'total_income' и 'days_employed'
- неверный формат в столбце'days_employed'
- дубликаты по регистру в столбце 'education' 
- общие дубликаты в таблице

**Вывод**

По результатам исследования были установлены зависимости по каждой из категорий, каждая из которых поможет оценить способность потенциального заёмщика вернуть кредит банку.